In [ ]:
import os
from neo.core import SpikeTrain
from quantities import ms, s
import numpy as np
import elephant
import viziphant
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
sorting_path = "/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Maxtwo/phornauer/Chemogenetics/Large_dose_range/well014/sorter_output/segment_0/"
spike_times = np.load(os.path.join(sorting_path,'spike_times.npy')).astype('float32')/10000
spike_templates = np.load(os.path.join(sorting_path,'spike_templates.npy'))

In [ ]:
t_cutoff = 30
cut_times = spike_times[spike_times < t_cutoff]
cut_units = spike_templates[spike_times < t_cutoff]

In [ ]:
unit_ids = np.unique(cut_units)
spike_train_list= [SpikeTrain(cut_times[cut_units==x]*s,t_stop=t_cutoff) for x in unit_ids]

In [ ]:
spike_trains = elephant.neo_tools.get_all_spiketrains(spike_train_list)

In [ ]:
kernel = elephant.kernels.GaussianKernel(10*ms)
fr_mat = elephant.statistics.instantaneous_rate(spike_trains,10*ms,kernel)

In [ ]:
viziphant.statistics.plot_instantaneous_rates_colormesh(fr_mat)


In [ ]:
from elephant.gpfa import GPFA


# specify fitting parameters
bin_size = 5 * ms
latent_dimensionality = 3

gpfa_3dim = GPFA(bin_size=bin_size, x_dim=latent_dimensionality)

In [ ]:
[spike_trains][0][0]

In [ ]:
trajectories_3d = gpfa_3dim.fit_transform([spike_trains])

In [ ]:
plt.close('all')

In [ ]:
plt.plot(trajectories_3d[0][0,], trajectories_3d[0][1,])
plt.show(True)

In [ ]:
binned_activity = np.histogram(spike_times[spike_times<30],6000)

In [ ]:
plt.plot(binned_activity[0])
plt.show()

In [ ]:
cmap = plt.get_cmap('viridis')

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
sc = ax.scatter(trajectories_3d[0][0,],trajectories_3d[0][1,],trajectories_3d[0][2,],c=range(6000),cmap=cmap)
fig.colorbar(sc)
plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot(trajectories_3d[0][0,],trajectories_3d[0][1,],trajectories_3d[0][2,])

plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(trajectories_3d[0][0,],trajectories_3d[0][1,])
#plt.xlim([-0.07,0.0])
#plt.ylim([-0.03,0.03])
plt.show()

In [ ]:
params = gpfa_3dim.params_estimated

In [ ]:
sorted_idx = np.argsort(params['C'][:,0])

In [ ]:
sorted_idx

In [ ]:
plt.pcolor(params['C'][sorted_idx,])
plt.show()